# KoBigBird_Keyword_Extraction_Frequancy_Weighted

기존 방법에 빈도수에 따른 Frequancy_Weightt를 반영하는 모델

## 본문 키워드 형태소 분석하여 후보 키워드 추출하기

형태소 분석기 komoran을 사용하기 위해 konlpy 설치

In [1]:
#!pip install konlpy

### 불용어 제거 함수 정의

In [1]:
def remove_stopwords(nouns_list, stopword_set):
#     for noun in nouns_list[:]: # 원소를 삭제하는 과정에서 누락이 발생하기 때문에 [:]를 통해 리스트의 복사본을 주어야 함.
#         if noun in stopword_set:
#             nouns_list.remove(noun)
    arr_removed = [n for n in nouns_list if n not in stopword_set]
    return arr_removed

영어 불용어 받아오기

In [2]:
from nltk.corpus import stopwords as eng_stopwords

eng_stopwords_set = set(eng_stopwords.words('english'))

### 영어 단어 명사 추출 함수 정의

In [3]:
import nltk

from string import punctuation
import re

def get_nouns_eng(text):
    # 명사 추출 시 url 제거
    url_pattern = re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    text = re.sub(url_pattern, '', text)
    url_pattern = re.compile(r"[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{2,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)")
    text = re.sub(url_pattern, '', text)
    
    text = text.encode('utf-8').decode('ascii', 'ignore')

    result = ""
    for t in text:
        if t not in punctuation:
            result += t

    text = result.strip()

    word_tokens = nltk.word_tokenize(text)
    tokens_pos = nltk.pos_tag(word_tokens)
    NN_words = []
    for word, pos in tokens_pos:
        if 'NN' in pos or 'NNP' in pos:
            NN_words.append(word)
            
    return NN_words

## 불용어(stopwords) 사전 정의

두 개의 한국어 불용어 사전을 이용

In [4]:
import csv

stopwords = []
with open('./data/stopwords/stopwords.txt', 'r') as f:
    for line in f:
        stopwords.append(line.rstrip('\n'))

with open('./data/stopwords/stopwords-ko.txt', 'r') as f:
    for line in f:
        stopwords.append(line.rstrip('\n'))
        
with open('./data/stopwords/stopwords-np.txt', 'r') as f:
    tsv_file = csv.reader(f, delimiter="\t")
    for line in tsv_file:
        stopwords.append(line[0])

stopwords_set = set(stopwords)
print(stopwords_set)

{'이천육', '저것', '어이', '남짓', '윙윙', '｜', '이럴정도로', '따르', '할지라도', '어디', '와르르', '점', '~', '경우', '한데', '대로 하다', '이때', '쉿', '바로', '위하', '”', '에게', '본', '하는것이 낫다', '６', '생각하', '월', '%', '까악', '＊', '만 못하다', '그리하여', '반드시', '대하여', '때가 되어', '얼마간', '쿵', '아니었다면', '잘', '안', '듯', '비걱거리다', '댕그', '타다', '하기는한데', '어째서', '구체적으로', '그럼에도 불구하고', '！', '여부', '해야한다', '드', '￥', '=', '해요', '둘', '만일', '보이', '외에도', '대하면', '＃', '쪽으로', '각종', '잇따라', '를', '마치', '조차', '때문', '으로써', '응', '졸졸', '그래서', '이천구', '이', '참나', '어느 년도', '번', '앞', '지든지', '한 이유는', ',', '\\', '않기 위해서', '게다가', '이 외에', '참', '8', '’', '이렇게되면', '‘', '요컨대', '한다면 몰라도', '과연', '쳇', '두', '이 되다', '각각', '여보시오', '되', '마저', '어쩔수 없다', '비교적', '휘익', '요만한걸', '모', '위에서 서술한바와같이', '아니', '칠', '하지마', '우리들', '바꾸어서 말하면', '딱', '그저', '근거로', '이 밖에', '하지마라', '하물며', '기타', '얼마', '그러면', '단지', '도달하다', '실로', '으로서', '2', '에 대해', '지말고', '매', '데', '할망정', '말하', '말할것도 없고', '그치지 않다', '하지 않도록', '문제', '하여금', '무엇때문에', '비추어 보아', '마음대로', '보드득', '중의하나', '헉헉', '곳', '후', '：', '로', '얼마 안 되는 것', '관련이 있

### 명사구 추출 함수 정의

In [36]:
import konlpy
import nltk
from konlpy.tag import Okt

# def get_nouns_phrase(text):
    
#     phrases = []
#     words = konlpy.tag.Komoran().pos(text)

#     # Define a chunk grammar, or chunking rules, then chunk
#     grammar = """
#     NP: {<N.*>*<Suffix>?}   # Noun phrase
#     """
#     parser = nltk.RegexpParser(grammar)
#     chunks = parser.parse(words)


#     print("\n# Print noun phrases only")
#     for subtree in chunks.subtrees():
#         if subtree.label()=='NP':
#             phrases.append(' '.join((e[0] for e in list(subtree))))
#     return phrases


def get_nouns_phrase(text):  
    okt = Okt()
    phrases = okt.phrases(text)
    return phrases

### 각 형태소 분석기별 명사 추출 함수 정의

형태소 분석을 통해 명사 추출, 불용어 제거, 영어 단어 추출하는 함수

In [6]:
from konlpy.tag import Mecab 
from konlpy.tag import Komoran
from konlpy.tag import Kkma
from konlpy.tag import Okt
from collections import Counter

def get_nouns_mecab(text, use_phrase=False):
    mecab = Mecab()
    nouns = mecab.nouns(text)
    if use_phrase:
        phrases = get_nouns_phrase(text)
        nouns.extend(phrases)
    eng_nouns = get_nouns_eng(text)
    eng_nouns = [ n for n in eng_nouns if len(n) != 1 ]
    eng_nouns = remove_stopwords(eng_nouns, eng_stopwords_set)
    nouns.extend(eng_nouns)
    nouns = remove_stopwords(nouns, stopwords_set)
    counts = Counter(nouns)
    nouns = set(nouns)
    nouns = list(nouns)
    
    return nouns, counts

def get_nouns_komoran(text, use_phrase=False):
    komoran = Komoran()
    nouns = komoran.nouns(text)
    if use_phrase:
        phrases = get_nouns_phrase(text)
        nouns.extend(phrases)
    eng_nouns = get_nouns_eng(text)
    eng_nouns = [ n for n in eng_nouns if len(n) != 1 ]
    eng_nouns = remove_stopwords(eng_nouns, eng_stopwords_set)
    nouns.extend(eng_nouns)
    nouns = remove_stopwords(nouns, stopwords_set)
    counts = Counter(nouns)
    nouns = set(nouns)
    nouns = list(nouns)
    
    return nouns, counts

def get_nouns_kkma(text):
    kkma = Kkma()
    nouns = kkma.nouns(text)
    nuons = remove_stopwords(nouns, stopwords_set)
    nouns = set(nouns)
    nouns = list(nouns)
    eng_nouns = get_nouns_eng(text)
    eng_nouns = [ n for n in eng_nouns if len(n) != 1 ]
    nouns.extend(eng_nouns)
    return nouns

def get_nouns_okt(text):
    okt = Okt()
    nouns = okt.nouns(text)
    nuons = remove_stopwords(nouns, stopwords_set)
    nouns = set(nouns)
    nouns = list(nouns)
    eng_nouns = get_nouns_eng(text)
    eng_nouns = [ n for n in eng_nouns if len(n) != 1 ]
    nouns.extend(eng_nouns)
    
    return nouns

In [7]:
# sample_nouns = get_nouns_komoran(sample[1]['body_text'])
# print(sample[1]['title'])
# print(sample[1]['body_text']+'\n')
# print(sample_nouns)

### 데이터 불러오기

In [8]:
import jsonlines

sample = []

with jsonlines.open("./data/brunch_NLI_test.jsonl") as f:
    for line in f.iter():
        if line['likes'] >= 180:
            sample.append(line)

In [9]:
len(sample)

217

### 문장to문장 sts finetuning된 KoBigBird 모델 불러오기

In [10]:
from transformers import  AutoModel,AutoTokenizer

#model = AutoModel.from_pretrained("monologg/kobigbird-bert-base", attention_type="original_full", add_pooling_layer=False)
#model = AutoModel.from_pretrained("./model/sts/saved_model_epoch_7.pt", attention_type="original_full", add_pooling_layer=False)
#model = AutoModel.from_pretrained("./model/nli/saved_model_epoch_2.pt", attention_type="original_full", add_pooling_layer=False)
#model = AutoModel.from_pretrained("./model/word_to_title/saved_model_epoch_3.pt", attention_type="original_full", add_pooling_layer=False)
#model = AutoModel.from_pretrained("./model/title_to_text/saved_model_epoch_2.pt", attention_type="original_full", add_pooling_layer=False)
model = AutoModel.from_pretrained("./model/word_to_text_fm3/saved_model_epoch_3.pt", attention_type="original_full", add_pooling_layer=False)

tokenizer = AutoTokenizer.from_pretrained("monologg/kobigbird-bert-base")

/home/user03/anaconda3/envs/mecab/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
import torch
import torch.nn as nn

device = "cuda" if torch.cuda.is_available() else "cpu"

model.to('cuda')
model.eval()

BigBirdModel(
  (embeddings): BigBirdEmbeddings(
    (word_embeddings): Embedding(32500, 768, padding_idx=0)
    (position_embeddings): Embedding(4096, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BigBirdEncoder(
    (layer): ModuleList(
      (0): BigBirdLayer(
        (attention): BigBirdAttention(
          (self): BigBirdSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BigBirdSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inp

## 최종 키워드 추출 함수(빈도수에 따른 가중치 반영)

In [83]:
def extract_main_keywords(nouns, counts, title, text, model, tokenizer, use_freq_w=False):
    
    tokenized_nouns = {}
    for keyword in nouns:
        tokenized_nouns[keyword] = tokenizer(keyword, max_length=32, padding='max_length', return_tensors='pt')
    
    keyword_vectors = {}
    for keyword, tokenized in tokenized_nouns.items():
        output = model(**tokenized.to(device))[0][:,0,:]
        keyword_vectors[keyword] = output
        
    tokenized_title = tokenizer(title, max_length=128, padding="max_length", return_tensors='pt')
    tokenized_text = tokenizer(text, max_length=4096, padding="max_length", return_tensors='pt')

    title_vec = model(**tokenized_title.to(device))[0][:,0,:]
    body_text_vec = model(**tokenized_text.to(device))[0][:,0,:]
    
    
    if use_freq_w:
        total_words_counts = 0
        for word, count in counts.items():
            if count != 1:
                total_words_counts += count
        print(total_words_counts)
        
        freq_w = {}
    
        for keyword in nouns:
            if counts[keyword] <= 1:
                freq_w[keyword] = 0
                continue
            freq_w[keyword] = (counts[keyword]/total_words_counts)
        
        print(freq_w)
        for keyword, vec in keyword_vectors.items():
            body_text_vec = ((freq_w[keyword]* vec) + body_text_vec)
        
        body_text_vec = body_text_vec/total_words_counts
        
    cos = nn.CosineSimilarity(dim=1, eps=1e-6)
    similarity = {}

    for keyword, vector in keyword_vectors.items():
        sim = cos(vector,  title_vec*(0.2) + body_text_vec*(0.8))
        similarity[keyword] = sim
        
    sorted_dict = sorted(similarity.items(), key = lambda item: item[1], reverse = True)
    
    # print(keyword_vectors)
    print(f"제목과 본문의 유사도 {cos(title_vec, body_text_vec)}\n")
    print(title+'\n')
    print(text+'\n')
    print(sorted_dict,'\n')
    print(nouns)
    
    return keyword_vectors, similarity

In [38]:
sample_essay = sample[6]

with torch.no_grad():
    print(sample_essay['keywords'])
    
    title = sample_essay['title']
    text = sample_essay['body_text']
    text = text.replace('\n','').replace('\t','').replace('\r','') # 코모란 에러 방지
    nouns, counts = get_nouns_mecab(text, use_phrase=True)
    print(counts)
    extract_main_keywords(nouns, counts, title, text, model, tokenizer, use_freq_w=False)

['워킹맘', '출근', '생각']
Counter({'마음': 10, '운전자': 7, '생각': 5, '업무': 5, '사람': 5, '양보': 5, '여유': 4, '도로': 4, '사고': 4, '출근길': 4, '감정': 4, '누군가': 4, '시작': 4, '길': 4, '배려': 4, '호의': 4, '상황': 3, '지각': 3, '순간': 3, '전날': 3, '마무리': 3, '노래': 3, '하루': 3, '사이': 3, '아침': 3, '사치': 3, '방향': 3, '이날': 3, '불편': 3, '게': 3, '행렬': 2, '사정': 2, '접촉': 2, '통행': 2, '신호': 2, '운전대': 2, '초보': 2, '조바심': 2, '테': 2, '프로젝트': 2, '머리': 2, '지도': 2, '회사': 2, '부모': 2, '죄책감': 2, '상태': 2, '라디오': 2, '고민': 2, '해소': 2, '주체': 2, '출발': 2, '내비게이션': 2, '도착': 2, '목록': 2, '재생': 2, '지시': 2, '차선': 2, '추월': 2, '차량': 2, '안전': 2, '담보': 2, '일지': 2, '월요일': 2, '족족': 2, '인지상정': 2, '테이크': 2, '회의': 2, '누가': 2, '상사': 2, '정적': 2, '평소': 2, '스트레스': 2, '요인': 2, '작용': 2, '작정': 2, '부담': 2, '감고': 2, '연습': 2, '내일': 2, '조선뉴스프레스': 2, '온라인': 2, '매체': 2, '건강': 2, '연재': 2, 'take': 2, '자동차': 1, '위': 1, '기민': 1, '방심': 1, '압박감': 1, '미안': 1, '부정': 1, '성공': 1, '정확': 1, '치열': 1, '무리': 1, '나라': 1, '워킹': 1, '맘': 1, '비상': 1, '기브': 1, '앤': 1, '고요': 1, '손': 1, '이기': 1, '날'

구현해야 할 점

* 단어를 배치단위로 입력하여 한번에 연산할 수 있게 변형
* 빈도수에 따른 가중치 변환
* 제목-본문 STS/NLI fine-tuning
* 단어-제목 STS/NLI fine-tuning
* 단어-본문 STS/NLI fine-tuning

### kr-wordrank와 비교

In [32]:
from krwordrank.word import KRWordRank

min_count = 1   # 단어의 최소 출현 빈도수 (그래프 생성 시)
max_length = 15 # 단어의 최대 길이
wordrank_extractor = KRWordRank(min_count=min_count, max_length=max_length)

beta = 0.85    # PageRank의 decaying factor beta
max_iter = 20
texts = [text]
keywords, rank, graph = wordrank_extractor.extract(texts, beta, max_iter)

for word, r in sorted(keywords.items(), key=lambda x:x[1], reverse=True)[:30]:
        print('%8s:\t%.4f' % (word, r))

      길을:	3.7722
   '마음건강:	3.6627
     사치를:	3.6603
   인지상정,:	3.4503
  연재합니다.:	3.3920
      켜지:	3.1892
      마음:	3.1291
      운전:	2.9968
     한다.:	2.9772
      않을:	2.9764
     시간이:	2.9577
      업무:	2.9411
      않는:	2.8918
      길게:	2.7575
    모른다.:	2.5993
길(mindgil.com)'에:	2.5838
     비집고:	2.5795
      일을:	2.5374
     했다.:	2.4885
      한껏:	2.4636
    당연하게:	2.3675
     감정이:	2.3459
     온라인:	2.3156
     행렬에:	2.3069
      말도:	2.3051
      있는:	2.3041
     하다.:	2.2575
     몰라도:	2.2342
    느긋함과:	2.2041
     나면…:	2.2019


## 명사구 추출 코드

In [16]:
komoran = Komoran()
pos = komoran.pos(text)
print(pos)

[('아침', 'NNG'), ('8', 'SN'), ('시', 'NNB'), ('쯤', 'NNB'), ('휴대폰', 'NNG'), ('알람', 'NNP'), ('소리', 'NNP'), ('에', 'JKB'), ('맞추', 'VV'), ('어', 'EC'), ('눈', 'NNG'), ('을', 'JKO'), ('뜨', 'VV'), ('ㄴ다', 'EF'), ('.', 'SF'), ('업무', 'NNG'), ('시간', 'NNG'), ('에', 'JKB'), ('제약', 'NNG'), ('이', 'JKS'), ('없', 'VA'), ('는', 'ETM'), ('프리랜서', 'NNP'), ('는', 'JX'), ('마냥', 'MAG'), ('늦', 'VA'), ('게', 'EC'), ('자', 'VV'), ('고', 'EC'), ('늦', 'VA'), ('게', 'EC'), ('일어나', 'VV'), ('아도', 'EC'), ('되', 'VV'), ('ㄹ', 'ETM'), ('걸', 'VV'), ('라고', 'EC'), ('생각하는 사람', 'NNP'), ('들', 'XSN'), ('이', 'JKS'), ('많', 'VA'), ('지만', 'EC'), (',', 'SP'), ('적어도', 'MAG'), ('고객', 'NNG'), ('들', 'XSN'), ('대부분', 'NNG'), ('이', 'JKS'), ('업무', 'NNG'), ('를', 'JKO'), ('하', 'VV'), ('는', 'ETM'), ('오전', 'NNP'), ('9', 'SN'), ('시', 'NNB'), ('부터', 'JX'), ('오후', 'NNG'), ('6', 'SN'), ('시', 'NNB'), ('까지', 'VV'), ('는', 'ETM'), ('제정신', 'NNG'), ('을', 'JKO'), ('차리', 'VV'), ('고', 'EC'), ('있', 'VV'), ('는', 'ETM'), ('편', 'NNB'), ('이', 'JKS'), ('좋', 'VA'), ('다는', 'ETM'

In [17]:
komoran = Komoran()
pos = komoran.pos(text)

np_list = []
np = ''
cnt = 0
for tup in pos:
    if 'N' in tup[1]:
        np = np + tup[0]
        cnt += 1
        
    elif np != '' and cnt != 1:
        np_list.append(np)
        np = ''
        cnt = 0
    else:
        np = ''
        cnt = 0

np_list

['아침8시쯤휴대폰알람소리',
 '업무시간',
 '생각하는 사람들',
 '고객들대부분',
 '오전9시',
 '오후6시',
 '업계선배들',
 '3년간의경험',
 '10분쯤',
 '전신스트레칭',
 '스틱커피',
 '일회용드립백',
 '캡슐형에스프레소머신',
 '기때문',
 '머그컵',
 '아침식사',
 '시간여유',
 '도보15분거리',
 '모닝세트',
 '해동해크림치즈',
 '아침식사',
 '식이다보니최근',
 '산지얼마',
 '뒤달력',
 '번체크',
 '작업중',
 '잡지번역',
 '교양과학서적',
 '3개월',
 '번씩발간',
 '인문계간지',
 '번역가5명',
 '남은커피',
 '번역일',
 '일러스트작업',
 '시간씩스케치',
 '포토샵으로채색',
 '고요한환경',
 '그릴때',
 '배꼽시계',
 '12시',
 '1시사이',
 '점심식사',
 '끼분량씩소분해',
 '제육볶음',
 '단백질반찬',
 '기때문',
 '오후작업시간',
 '프리랜서생활초기',
 '종일집중',
 '사람구경',
 '겸노트북',
 '집앞카페',
 '오후1시경',
 '가도주변회사',
 '직장인들',
 '30분',
 '손님들',
 '아기엄마',
 '학생들사이에서나',
 '음악소리',
 '데방해',
 '배경음악',
 '주변사람들',
 '기때문',
 '평일낮',
 '씩친구',
 '회사앞',
 '점심약속',
 '조조영화',
 '저녁시간',
 '지인들',
 '내몫이다',
 '한강둔치',
 '이때자리',
 '내몫이다',
 '저녁식사',
 '후남은작업',
 '저녁약속',
 '취미생활',
 '손바느질',
 '기때문',
 '그리워질때',
 '동네친구들',
 '때상경',
 '학교근처',
 '기때문',
 '친구들',
 '명쯤',
 '뒤침대',
 '하나씩',
 '내일아침눈',
 '라고대답',
 '구체적',
 '이것들',
 '내일',
 '4년전',
 '사무직회사원',
 '3년전',
 '마음하나로기술하나',
 '백수',
 '작가인스타그램',
 '작가유튜브',
 '서메리']

In [18]:
import konlpy
import nltk

# POS tag a sentence
sentence = u'만 6세 이하의 초등학교 취학 전 자녀를 양육하기 위해서는'
words = konlpy.tag.Twitter().pos(text)

# Define a chunk grammar, or chunking rules, then chunk
grammar = """
NP: {<N.*>*}   # Noun phrase
"""
parser = nltk.RegexpParser(grammar)
chunks = parser.parse(words)


print("\n# Print noun phrases only")
for subtree in chunks.subtrees():
    if subtree.label()=='NP':
        print(' '.join((e[0] for e in list(subtree))))


/home/user03/anaconda3/envs/mecab/lib/python3.9/site-packages/konlpy/tag/_okt.py:17: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')



# Print noun phrases only
아침 8시
휴대폰 알람 소리
눈
업무 시간
제약
프리랜서
마냥
자고
거
생각
사람
고객
대부분
업무
오전 9시
오후 6시
정신
편이
것
업계 선배
조언
지난 3년
경험
원칙
일어나서 가장 먼저
일
스트레칭
가장 먼저 화장실
그 다음
10분
꼬박
전신 스트레칭
밤사이
근육
풀
정신
좀
후
비타민
물
컵
커피
린다
줄곧 스틱 커피
일회용 드립
이용
얼마 전
맘
캡슐
에스프레소 머신
구입
뒤
더 제대로
아메리카노
수
커피
카페인
체질
밤
잠
수
때문
머그컵
에스프레소
샷
물
가득 부어
편이
아침식사
빵
애용
가끔 시간 여유
때
집
도보 15분 거리
패스트푸드 점
모닝 세트
가기
대개
집
커피
빵
프라이팬
식빵
잼
발라
때
냉동 실
베이글
해동
크림 치즈
발라
때
아침 식사
대개
식이
최근
토스터
욕심
에스프레소 머신
산지 얼마 안
당분간
식사
마치
세수
양치
뒤 달력
오늘
일정
다시
번 체크
오늘
요즘 한창 작업 중인 단행본
잡지 번역
예정
단행본
지구력
교양 과학 서적
처음
끝
내
번역
작품
잡지
3 개월
번
발간
철학
인문 계간지
현재 번역가 5 명
투입
공역
일
확인
후
자료
노트북
커피
책상
오전 내내
종종
화장실
물
간식
기도
작업
지금
번역 일
집중
일러스트 작업
때
하루
몇 시간
스케치
거나 포토샵
채색
번역
때
텍스트
위해 고요한 환경
일
편이
그림
그릴 때
음악
를
기도
배꼽 시계
편이
12시
1시 사이
집중
점심식사
주로 밥
번
끼 분량
분해
밥
해동
냄비
국
냉장고
반찬
국
찌개
대개 내
직접
밑반찬
고향
엄마
경우
여기
냉장고
재료
계란말이
볶음
단백질 반찬
하나
밥
직장인
마찬가지
프리랜서
점심
일과 시간
낙 이기 때문
때
시간
정성 스레
노력
식사
설거지
마치
다시 오후 작업시간
군기
바짝
프리랜서 생활 초기
집
종일 집중
최근
배
오후
집중
날
종종
때
분위기
환기
사람 구경
겸 노트북
집 앞 카페
오후 1시
카페
가도 주변 회사
직장인
발
틈
북적
30분
손님
썰물
거짓말
내
것
바로 이 

In [19]:
print(words)

[('아침', 'Noun'), ('8시', 'Number'), ('쯤', 'Foreign'), ('휴대폰', 'Noun'), ('알람', 'Noun'), ('소리', 'Noun'), ('에', 'Josa'), ('맞춰', 'Verb'), ('눈', 'Noun'), ('을', 'Josa'), ('뜬다', 'Verb'), ('.', 'Punctuation'), ('업무', 'Noun'), ('시간', 'Noun'), ('에', 'Josa'), ('제약', 'Noun'), ('이', 'Josa'), ('없는', 'Adjective'), ('프리랜서', 'Noun'), ('는', 'Josa'), ('마냥', 'Noun'), ('늦게', 'Verb'), ('자고', 'Noun'), ('늦게', 'Verb'), ('일어나도', 'Verb'), ('될', 'Verb'), ('거', 'Noun'), ('라고', 'Josa'), ('생각', 'Noun'), ('하는', 'Verb'), ('사람', 'Noun'), ('들', 'Suffix'), ('이', 'Josa'), ('많지만', 'Adjective'), (',', 'Punctuation'), ('적어도', 'Adverb'), ('고객', 'Noun'), ('들', 'Suffix'), ('대부분', 'Noun'), ('이', 'Josa'), ('업무', 'Noun'), ('를', 'Josa'), ('하는', 'Verb'), ('오전', 'Noun'), ('9시', 'Number'), ('부터', 'Foreign'), ('오후', 'Noun'), ('6시', 'Number'), ('까지는', 'Josa'), ('제', 'Modifier'), ('정신', 'Noun'), ('을', 'Josa'), ('차리고', 'Verb'), ('있는', 'Adjective'), ('편이', 'Noun'), ('좋다는', 'Adjective'), ('것', 'Noun'), ('이', 'Josa'), ('업계', 'Noun'), ('선배', '

## OKT 메서드를 활용한 명사구 추출

In [34]:
from konlpy.tag import Okt

okt = Okt()
okt.phrases(text)

['자동차들',
 '자동차들의 행렬',
 '오도',
 '오도 가도',
 '오도 가도 못',
 '오도 가도 못하는 상황',
 '지각',
 '여유',
 '도로',
 '도로 위',
 '도로 위 사정',
 '시시각각',
 '바쁜 마음',
 '접촉사고',
 '생각',
 '꽉 막힌 도로',
 '수 있는 거',
 '통행',
 '통행 신호',
 '것뿐',
 '운전대',
 '2년',
 '채 안 된 초보',
 '채 안 된 초보 운전자',
 '방심하는 순간',
 '사고',
 '출근길',
 '온갖',
 '온갖 감정',
 '누군가',
 '전날',
 '전날 마무리',
 '업무',
 '업무 생각',
 '조바심',
 '앞서',
 '앞서갔을 테',
 '또 누군가',
 '시작',
 '시작할 프로젝트',
 '인한 압박감',
 '머리',
 '지끈거릴지도',
 '아이',
 '회사',
 '부모',
 '죄책감',
 '예민한 상태일',
 '예민한 상태일 수도',
 '감정들',
 '있는 건',
 '라디오',
 '노래',
 '하루',
 '시작하는 사람',
 '고민',
 '바로',
 '바로 전날',
 '바로 전날 해소',
 '홀가분한 이도',
 '공들',
 '일이',
 '마무리',
 '들뜬 마음',
 '주체',
 '못하는 운전자',
 '있을 테',
 '그들',
 '그들 사이',
 '아침',
 '사치',
 '부린다',
 '조금',
 '출발',
 '전 내비게이션',
 '도착',
 '도착 시간',
 '살피',
 '좋아하는 노래',
 '좋아하는 노래 목록',
 '재생',
 '방향',
 '시등',
 '차선',
 '추월',
 '추월하는 운전자',
 '예사',
 '차량',
 '운전자',
 '적지',
 '사람',
 '안전',
 '담보',
 '시간',
 '그런 사람들',
 '우리나라',
 '누구',
 '지지',
 '지지 않는 워킹맘',
 '양보',
 '여유로움',
 '어쩌면',
 '어쩌면 나를 위',
 '배려일지',
 '월요일',
 '이날',
 '심기',
 '족족',
 '비상등',
 '비상등 한번',
 '인지상정',
 '기

In [37]:
get_nouns_phrase(text)

['자동차들',
 '자동차들의 행렬',
 '오도',
 '오도 가도',
 '오도 가도 못',
 '오도 가도 못하는 상황',
 '지각',
 '여유',
 '도로',
 '도로 위',
 '도로 위 사정',
 '시시각각',
 '바쁜 마음',
 '접촉사고',
 '생각',
 '꽉 막힌 도로',
 '수 있는 거',
 '통행',
 '통행 신호',
 '것뿐',
 '운전대',
 '2년',
 '채 안 된 초보',
 '채 안 된 초보 운전자',
 '방심하는 순간',
 '사고',
 '출근길',
 '온갖',
 '온갖 감정',
 '누군가',
 '전날',
 '전날 마무리',
 '업무',
 '업무 생각',
 '조바심',
 '앞서',
 '앞서갔을 테',
 '또 누군가',
 '시작',
 '시작할 프로젝트',
 '인한 압박감',
 '머리',
 '지끈거릴지도',
 '아이',
 '회사',
 '부모',
 '죄책감',
 '예민한 상태일',
 '예민한 상태일 수도',
 '감정들',
 '있는 건',
 '라디오',
 '노래',
 '하루',
 '시작하는 사람',
 '고민',
 '바로',
 '바로 전날',
 '바로 전날 해소',
 '홀가분한 이도',
 '공들',
 '일이',
 '마무리',
 '들뜬 마음',
 '주체',
 '못하는 운전자',
 '있을 테',
 '그들',
 '그들 사이',
 '아침',
 '사치',
 '부린다',
 '조금',
 '출발',
 '전 내비게이션',
 '도착',
 '도착 시간',
 '살피',
 '좋아하는 노래',
 '좋아하는 노래 목록',
 '재생',
 '방향',
 '시등',
 '차선',
 '추월',
 '추월하는 운전자',
 '예사',
 '차량',
 '운전자',
 '적지',
 '사람',
 '안전',
 '담보',
 '시간',
 '그런 사람들',
 '우리나라',
 '누구',
 '지지',
 '지지 않는 워킹맘',
 '양보',
 '여유로움',
 '어쩌면',
 '어쩌면 나를 위',
 '배려일지',
 '월요일',
 '이날',
 '심기',
 '족족',
 '비상등',
 '비상등 한번',
 '인지상정',
 '기

## 클러스터링 한 후 최종 단어 추출

In [48]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
  
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import normalize
from sklearn.decomposition import PCA


def cluster_dbscan(vec_dict):
    word_features_numpy = {}
    for word, feat in vec_dict.items():
        word_features_numpy[word] = feat.to('cpu')[0].numpy()
        
    features_df = pd.DataFrame(word_features_numpy)
    features_df = features_df.transpose()
    
    clustering = DBSCAN(eps=8.4, min_samples=2).fit(features_df)
    DBSCAN_dataset = features_df.copy()
    DBSCAN_dataset.loc[:,'Cluster'] = clustering.labels_ 
    
    print(DBSCAN_dataset.Cluster.value_counts().to_frame())
    print(clustering.labels_ .max())

In [84]:
with torch.no_grad():
    vec_dict, sim_dict = extract_main_keywords(nouns, counts, title, text, model, tokenizer, use_freq_w=False)

제목과 본문의 유사도 tensor([0.8596], device='cuda:0')

내일은 양보 안 합니다

길게 늘어선 자동차들의 행렬에 끼어있었다. 오도 가도 못하는 상황. 지각을 면하려면 서둘러야 한다. 여유 있게 집을 나섰는데, 도로 위 사정은 시시각각 변했다. 바쁜 마음에 서둘렀다가 접촉사고라도 나면… 생각하기도 싫다. 조급함을 내려놓아야 했다. 꽉 막힌 도로에서 할 수 있는 거라고는 통행 신호가 떨어졌을 때 기민하게 움직이는 것뿐이다. 운전대를 잡은 지 2년이 채 안 된 초보 운전자가 방심하는 순간, 사고로 이어진다.      출근길에는 온갖 감정이 뒤엉켜 뛰어다닌다. 누군가는 전날 마무리하지 못한 업무 생각에 조바심이 저만치 앞서갔을 테고, 또 누군가는 새로 시작할 프로젝트로 인한 압박감에 머리가 지끈거릴지도 모른다. 우는 아이를 떼어놓고 회사에 가는 부모는 미안함과 죄책감으로 한껏 예민한 상태일 수도 있다. 부정적인 감정들만 있는 건 아니다. 라디오에서 흘러나오는 노래를 따라부르며 하루를 즐겁게 시작하는 사람도, 오랫동안 괴롭히던 고민이 바로 전날 해소돼 홀가분한 이도, 공들이던 일이 성공적으로 마무리돼 한껏 들뜬 마음을 주체하지 못하는 운전자도 있을 테다. 그들 사이에 나도 있다.      나는 아침마다 사치를 부린다. 정확하게 말하면, 아이가 조금 자라면서 출근길에 여유를 부리기 시작했다. 출발하기 전 내비게이션에 뜬 도착 시간을 슬쩍 살피고, 좋아하는 노래 목록을 재생한다. 꽉 막힌 도로에 나오면 치열하기만 하다. 방향지시등을 켜지 않고 차선을 넘어 추월하는 운전자는 예사고, 줄지어 기다리는 차량을 비집고 무리하게 끼어드는 운전자도 적지 않다. 아침에 바쁘지 않은 사람은 없으련만, 안전을 담보로 시간을 버는 듯하다.      그런 사람들에게 난 길을 내어준다. 우리나라에서 바쁘기로 따지면 누구에게도 지지 않는 워킹맘이지만, ‘양보’라는 사치를 부려본다. 하루가 어떻게 흘러갈지는 몰라도 출근길만큼은 느긋함과 여유로움을 즐기고 싶은, 어쩌면 나

In [117]:
word_features_numpy = {}
for word, feat in vec_dict.items():
    word_features_numpy[word] = feat.to('cpu')[0].numpy()

features_df = pd.DataFrame(word_features_numpy)
features_df = features_df.transpose()

# 코사인 유사도 부착

clustering = DBSCAN(eps=6, min_samples=2).fit(features_df)
DBSCAN_dataset = features_df.copy()
DBSCAN_dataset.loc[:,'Cluster'] = clustering.labels_ 

print(DBSCAN_dataset.Cluster.value_counts().to_frame())
print(clustering.labels_ .max())

     Cluster
-1        98
 6        29
 2        22
 3         6
 5         5
 9         5
 19        5
 18        4
 10        4
 12        4
 26        3
 23        3
 22        3
 20        3
 15        3
 4         3
 13        3
 1         3
 14        2
 29        2
 28        2
 27        2
 25        2
 24        2
 7         2
 0         2
 21        2
 8         2
 11        2
 17        2
 16        2
 30        2
30


In [99]:
DBSCAN_dataset

,0,1,2,3,4,5,6,7,8,9,...,759,760,761,762,763,764,765,766,767,Cluster
누가,0.720863,0.114998,0.254447,0.338540,-0.178289,0.147485,0.949798,-0.316082,-0.446251,-0.221504,...,-0.659095,0.830374,0.712776,-0.249442,0.266249,0.029445,0.100356,-0.368535,0.332507,-1
재생,-0.044108,1.053724,0.324032,1.134930,0.102087,0.433873,0.353040,-0.359511,0.207214,-0.276842,...,-0.110791,0.578156,-0.357895,-0.168127,-0.546964,-0.308504,0.192078,-0.320386,0.620639,-1
연재,0.198611,-0.184315,0.186140,0.074648,0.058537,1.184786,0.869992,-0.477235,-0.635491,-0.147705,...,-0.031470,0.395314,0.220410,-0.617684,-0.723010,0.076657,-0.264539,-0.481093,-0.618999,-1
호의,0.234873,-0.080860,0.465018,-0.503519,-0.138966,-0.289422,0.425529,-0.397129,-0.922399,-0.087800,...,0.160151,0.203226,-0.114936,-0.134378,0.154226,-0.187244,0.011680,-0.577554,0.272208,0
지끈거릴지도,0.500123,-0.273957,0.146948,-0.082898,0.015909,0.353336,0.044573,-0.337228,-0.686075,-0.078519,...,-0.444981,0.544274,0.198182,0.145524,0.280052,-0.235639,0.070835,-0.219943,0.149195,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
인지상정,0.881213,0.093503,0.292376,0.283616,0.133389,0.557739,0.292888,-0.128249,-1.397652,-0.133442,...,0.219033,0.745053,-0.108808,0.212369,0.016729,-0.968727,0.186279,-0.466870,0.162159,6
추월,0.534988,0.413689,0.313163,-0.374431,0.028111,0.354399,0.800948,-0.089184,-0.726909,0.012070,...,-0.132631,1.097829,-0.029354,-0.416831,0.942720,0.382073,-0.023050,-0.222061,0.900091,0
상황,1.048259,-0.279629,0.305398,-0.055119,0.156039,-0.161587,-0.356807,0.168842,-0.171856,-0.025086,...,-0.323365,0.209764,-0.319298,-0.040979,0.941835,-0.379201,-0.029021,-0.182795,-0.100796,0
상태,0.810598,-0.150887,0.382932,0.032356,0.147430,0.142625,-0.348224,0.046241,-0.599488,-0.127144,...,-0.432539,0.472674,0.057335,-0.012186,0.128727,-0.517450,-0.215671,-0.415064,0.007296,0


### 키(각 단어)에 대하여 코사인 유사도 컬럼 추가

In [125]:
#sim_dict
# 데이터 프레임의 키 받아오기
sim_list = []
for word in DBSCAN_dataset.index:
    sim_list.append(sim_dict[word].item())
sim_list

[0.7115219235420227,
 0.31035104393959045,
 0.31796446442604065,
 0.5878422260284424,
 0.6766993999481201,
 0.6692169904708862,
 0.1694764643907547,
 0.7046090364456177,
 0.6864705085754395,
 0.7049600481987,
 0.8035280108451843,
 0.7608306407928467,
 0.7493561506271362,
 0.754081130027771,
 0.7421118021011353,
 0.5940326452255249,
 0.4951702356338501,
 0.6241728663444519,
 0.6220680475234985,
 0.6720196008682251,
 0.6936262845993042,
 0.6649662256240845,
 0.7547933459281921,
 0.6538593769073486,
 0.6960622072219849,
 0.6536332368850708,
 0.7964155673980713,
 0.4788302779197693,
 0.5395662784576416,
 0.6928068399429321,
 0.6444050073623657,
 0.7778164744377136,
 0.8226296305656433,
 0.7727175951004028,
 0.5278900861740112,
 0.7437719702720642,
 0.6014487147331238,
 0.7718043923377991,
 0.620366096496582,
 0.5933946967124939,
 0.6005468368530273,
 0.3748847246170044,
 0.700636088848114,
 0.7227624654769897,
 0.7241855263710022,
 0.39170926809310913,
 0.7246310114860535,
 0.5985667705535

In [126]:
DBSCAN_dataset['sim'] = sim_list
DBSCAN_dataset

,0,1,2,3,4,5,6,7,8,9,...,760,761,762,763,764,765,766,767,Cluster,sim
누가,0.720863,0.114998,0.254447,0.338540,-0.178289,0.147485,0.949798,-0.316082,-0.446251,-0.221504,...,0.830374,0.712776,-0.249442,0.266249,0.029445,0.100356,-0.368535,0.332507,-1,0.711522
재생,-0.044108,1.053724,0.324032,1.134930,0.102087,0.433873,0.353040,-0.359511,0.207214,-0.276842,...,0.578156,-0.357895,-0.168127,-0.546964,-0.308504,0.192078,-0.320386,0.620639,-1,0.310351
연재,0.198611,-0.184315,0.186140,0.074648,0.058537,1.184786,0.869992,-0.477235,-0.635491,-0.147705,...,0.395314,0.220410,-0.617684,-0.723010,0.076657,-0.264539,-0.481093,-0.618999,-1,0.317964
호의,0.234873,-0.080860,0.465018,-0.503519,-0.138966,-0.289422,0.425529,-0.397129,-0.922399,-0.087800,...,0.203226,-0.114936,-0.134378,0.154226,-0.187244,0.011680,-0.577554,0.272208,0,0.587842
지끈거릴지도,0.500123,-0.273957,0.146948,-0.082898,0.015909,0.353336,0.044573,-0.337228,-0.686075,-0.078519,...,0.544274,0.198182,0.145524,0.280052,-0.235639,0.070835,-0.219943,0.149195,-1,0.676699
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
인지상정,0.881213,0.093503,0.292376,0.283616,0.133389,0.557739,0.292888,-0.128249,-1.397652,-0.133442,...,0.745053,-0.108808,0.212369,0.016729,-0.968727,0.186279,-0.466870,0.162159,14,0.632594
추월,0.534988,0.413689,0.313163,-0.374431,0.028111,0.354399,0.800948,-0.089184,-0.726909,0.012070,...,1.097829,-0.029354,-0.416831,0.942720,0.382073,-0.023050,-0.222061,0.900091,6,0.842726
상황,1.048259,-0.279629,0.305398,-0.055119,0.156039,-0.161587,-0.356807,0.168842,-0.171856,-0.025086,...,0.209764,-0.319298,-0.040979,0.941835,-0.379201,-0.029021,-0.182795,-0.100796,-1,0.799209
상태,0.810598,-0.150887,0.382932,0.032356,0.147430,0.142625,-0.348224,0.046241,-0.599488,-0.127144,...,0.472674,0.057335,-0.012186,0.128727,-0.517450,-0.215671,-0.415064,0.007296,-1,0.732616


### 각 클러스터에서 유사도가 높은 1위만 뽑기

In [127]:
DBSCAN_dataset[DBSCAN_dataset['Cluster']==0]

,0,1,2,3,4,5,6,7,8,9,...,760,761,762,763,764,765,766,767,Cluster,sim
호의,0.234873,-0.080860,0.465018,-0.503519,-0.138966,-0.289422,0.425529,-0.397129,-0.922399,-0.087800,...,0.203226,-0.114936,-0.134378,0.154226,-0.187244,0.011680,-0.577554,0.272208,0,0.587842
안 될 호의,0.429511,-0.147812,0.410346,-0.404944,-0.112261,-0.232464,0.152825,-0.369834,-0.985462,-0.163609,...,-0.046512,-0.075639,0.090911,0.124477,-0.410564,0.023694,-0.491326,0.100529,0,0.569815


In [128]:
# 1. 전체 클러스터 리스트로 받아오기

cluster_list = DBSCAN_dataset.Cluster.to_list()

cluseter_set = set(cluster_list)
cluster_list = list(cluseter_set)
cluster_list

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 -1]

In [129]:
DBSCAN_dataset[DBSCAN_dataset['Cluster']==7]

,0,1,2,3,4,5,6,7,8,9,...,760,761,762,763,764,765,766,767,Cluster,sim
방심,0.783082,0.028129,0.235143,-0.001985,-0.044456,0.385911,-0.243009,-0.318193,-1.117089,-0.053608,...,0.127716,-0.209888,0.201597,0.314793,-0.258448,-0.271752,-0.327430,0.238534,7,0.742112
방심하는 순간,0.882303,0.011661,0.326331,-0.022353,-0.154452,0.447273,-0.086898,-0.182228,-1.072944,-0.006655,...,0.291491,-0.195044,0.000580,0.404619,-0.160294,-0.263110,-0.337048,0.137301,7,0.778352


In [153]:
# 잡음(이상치)인 -1 레이블을 제외하고 각 클러스터에서 가장 sim이 높은 단어 인덱스 반환
selected_words = []

for cluster in cluster_list:
    if cluster == -1:
        df = DBSCAN_dataset[DBSCAN_dataset['Cluster']==cluster]
        #print(df.index.to_list())
        selected_words.extend(df.index.to_list())
    else:
        df = DBSCAN_dataset[DBSCAN_dataset['Cluster']==cluster]
        df = df.sort_values('sim', ascending=False)
        top_word = df.iloc[0].name
        selected_words.append(top_word)

print(selected_words)
print(len(selected_words))

['호의', '조선뉴스프레스 온라인 매체', '업무 스트레스의 또', '조바심', '죄책감', '불편한 게', '차들 사이', '방심하는 순간', '손', '바로 전날 해소', '그 시간', '좋아하는 노래', '고민', '있을 테', '인지상정', '기브', '회의 시간', '어쩌면 나를 위', '양보', '오도 가도 못하는 상황', '사이', '사람들', '지지 않는 워킹맘', '채 안 된 초보', '있는 건', '예민한 상태일', '압박감', '겁', '비상등 한번', '누군가', '시작할 프로젝트', '누가', '재생', '연재', '지끈거릴지도', '기민', '예사', '업무량', '방향', '시등', '홀가분한 이도', '안전', '이기', '미안', '작용', '사정', '감고', '살피', '온갖', '길', '아침', '담보', '적지', '2년', '족족', '불편한 게 마뜩하', '사치', '마뜩하', 'give', '나라', '부린다', '주지', '정적', '지지', '공들', '위', '중요', '게', '접촉', '심기', '로움', '수도', '건강', '무리', '부정', '순간', '치열', '지도', '주체', '워킹', '라디오', '이도', '시시각각', '이날', '한번', '앤', '출근길', '평소', '우리나라', '상할 수 있는 상황', '것뿐', '지레', '작정', '지시', '일지', 'take', '비상', '불쑥', '주지 않을 작정', '신호', '눈', '빠듯했', '테이크', '앞서', '부담', '가끔', '글', '사고', '배려일지', '무량', '위험', '요인', '거릴', '일이', '량', '채 안 된 초보 운전자', '이들', '매체', '해소', '온라인', '접촉사고', '지끈', '이기적', '그래야', '목록', '못 견디는 사람', '좋아하는 노래 목록', '상황', '상태']
129


In [159]:
selected_words_dict = {}

for word in selected_words:
    selected_words_dict[word] = sim_dict[word]
sorted_selected_words = sorted(selected_words_dict.items(), key = lambda item: item[1], reverse = True)
sorted_selected_words[:10]

[('차들 사이', tensor([0.8820], device='cuda:0')),
 ('출근길', tensor([0.8688], device='cuda:0')),
 ('양보', tensor([0.8627], device='cuda:0')),
 ('상할 수 있는 상황', tensor([0.8261], device='cuda:0')),
 ('배려일지', tensor([0.8145], device='cuda:0')),
 ('접촉', tensor([0.8137], device='cuda:0')),
 ('사이', tensor([0.8107], device='cuda:0')),
 ('못 견디는 사람', tensor([0.8062], device='cuda:0')),
 ('조바심', tensor([0.8035], device='cuda:0')),
 ('예민한 상태일', tensor([0.8026], device='cuda:0'))]